In [125]:
# importazione delle librerie
import pandas as pd
import pandas_ta as ta
import numpy as np
import funzioni as fn
import matplotlib.pyplot as plt
from pprint import pprint
from itertools import combinations, permutations

# importiamo il path
import os
from funzioni import PATH, PATH_ESTR

CREIAMO LA NOSTRA CLASSE PER IL GIOCO DEL LOTTO

In [51]:
# funzione per dire quando e' uscito un numero
def uscite(df, numero):
    # prendiamo l'indice della riga che contiene un valore in una determinata colonna
    uscite = np.array([])
    for i in df.columns[2:]:
        # aggiungi l'indice della riga che contiene il valore 58 nella colonna i
        uscite = np.append(uscite, df[df[i] == str(numero)].index.values.astype(int))
    # aggiungiamo ad ogni valore della lista il valore 1
    uscite = np.sort(uscite + 1)

    return uscite


In [89]:
def ritardi(df, numero):

    u = uscite(df, numero)
    # calcuiamo la differenza tra ogni valore della lista e il successivo
    ritardi = np.diff(u)
    # inseriamo al primo posto il valore 0
    ritardi = np.insert(ritardi, 0, 0)

    return ritardi

In [135]:
class Lotto:

    def __init__(self, path=PATH_ESTR):
        """Di default il path e' la cartella estrazioni"""
        self.ruota = None
        self.df = None
        self.numero = None
        self.path = path

    def carica_estrazioni(self, ruota):
        r = self.path+ruota+'.csv'
        estrazioni = fn.leggi_da_csv(r)
        df = pd.DataFrame(estrazioni[1:], columns=estrazioni[0])
        return df

    def carica_numero(self, numero, ruota):

        self.ruota = ruota
        self.df = self.carica_estrazioni(self.ruota)
        self.numero = numero
        
        # calcoliamo le posizioni di indice delle uscite del numero
        self.elenco_uscite = uscite(self.df, self.numero)
        # il numero totali delle volte che e' uscito e' dato dalla lunghezza della lista
        self.n_uscite = len(self.elenco_uscite)
        # il totale delle estrazioni e' dato dalla lunghezza del dataframe
        self.totale_estrazioni = len(self.df)
        # il vettore delle frequenze e' dato dal vettore del numero di uscite diviso il vettore dell'indice del vettore stesso + 1
        self.frequenze = self.elenco_uscite / (np.arange(len(self.elenco_uscite))+1)
        # la frequenza media (ultima frequenza o frequenza attuale) e' data dal totale delle estrazioni diviso il numero di uscite
        self.ultima_frequenza = round(float(self.totale_estrazioni/self.n_uscite), 2)
        # l'elenco dei ritardi e' dato dal vettore dei ritardi
        self.ritardi = ritardi(self.df, self.numero)
        # il ritardo attuale e' dato dalla differenza tra il totale delle estrazioni e l'indice l'ultima uscita
        self.ritardo_attuale = self.totale_estrazioni - self.elenco_uscite[-1]
        # calcoliamo il ritardo massimo storico
        self.ritardo_massimo = self.ritardi.max()
        # salviamo l'ultimo ritardo
        self.ultimo_ritardo = self.ritardi[-1]
        # calcoliamo la scompensazione
        self.scompensazioni = self.ritardi - self.frequenze
        # calcoliamo la scompensazione attuale e l'ultima scompensazione
        self.ultima_scompensazione = self.scompensazioni.sum()
        self.scompensazione_attuale = self.ultima_scompensazione + (self.ritardo_attuale - self.ultima_frequenza)
        # salviamo le statistiche in un dizionario
        self.statistiche = {
            'ruota': self.ruota,
            'numero': self.numero,
            'n_uscite': self.n_uscite,
            'totale_estrazioni': self.totale_estrazioni,
            'ultima_frequenza': self.ultima_frequenza,
            'ritardo_attuale': self.ritardo_attuale,
            'ritardo_massimo': self.ritardo_massimo,
            'ultimo_ritardo': self.ultimo_ritardo,
            'ultima_scompensazione': self.ultima_scompensazione,
            'scompensazione_attuale': self.scompensazione_attuale
        }
    def stampa_statistihe(self):
        
        for k, v in self.statistiche.items():
            print(f'{k}: {v}')


In [132]:
# leggiamo il file csv delle estrazioni di Bari
a = Lotto()

In [133]:
a.carica_numero(58, 'BA')
a.stampa_statistihe()

ruota: BA
numero: 58
n_uscite: 385
totale_estrazioni: 6723
ultima_frequenza: 17.46
ritardo_attuale: 5.0
ritardo_massimo: 115.0
ultima_scompensazione: 185.5348877738894
scompensazione_attuale: 173.0748877738894


In [136]:
b = Lotto(path=PATH)
b.carica_numero(58, 'temp')
print(b.ultimo_ritardo, b.ultima_scompensazione)


9.0 185.5348877738894


In [192]:
import time
n = Lotto(path=PATH)

bari = fn.leggi_da_csv(PATH_ESTR+'BA.csv')
df_bari = pd.DataFrame(bari[1:], columns=bari[0])
# creiamo un array a 2 dimensioni
t = []
for i in range(50, len(df_bari)):
    # eliminare il file temp.csv
    os.remove(PATH+'temp.csv')
    p = df_bari[:i]
    p.set_index('Data', inplace=True)
    p.to_csv(PATH+'temp.csv')
    temp = fn.leggi_da_csv(PATH+'temp.csv')
    df_temp = pd.DataFrame(temp[1:], columns=temp[0])
    
    # prendiamo i valori delle ultime 5 colonne dell'ultima riga
    l = list(df_temp.iloc[-1].values)
    for i in df_temp.iloc[-1, 2:].values:
        n.carica_numero(i, 'temp')
        l.append(n.ultimo_ritardo)
        l.append(n.ultima_scompensazione)
    
    # aggiungiamo alla matrice t la riga l
    t.append(l)

# trasformiamo la matrice t in un dataframe
df = pd.DataFrame(t, columns=['Data', 'Ruota', '1', '2', '3', '4', '5', '2_ritardo', '2_scompensazione', '3_ritardo', '3_scompensazione', '4_ritardo', '4_scompensazione', '5_ritardo', '5_scompensazione'])
df.to_csv(PATH+'dati.csv')


KeyboardInterrupt: 

capire come semplicificare la classe per renderla csv indipendente
semplificare il loop per fare il dataframe con le scomp e i rit